In [3]:
import json
from pprint import pprint

In [4]:
!pwd 

/Users/katychow/Projects/detect_sarcasm


In [5]:
sarcasm = '/Users/katychow/Projects/detect_sarcasm/Sarcasm_Headlines_Dataset.json'

In [6]:
data = []
with open('Sarcasm_Headlines_Dataset.json') as f:
    for line in f:
        data.append(json.loads(line))

In [7]:
data[0].values(), data[0].keys()

(dict_values(['https://www.huffingtonpost.com/entry/versace-black-code_us_5861fbefe4b0de3a08f600d5', "former versace store clerk sues over secret 'black code' for minority shoppers", 0]),
 dict_keys(['article_link', 'headline', 'is_sarcastic']))

In [8]:
data[2]['article_link']

'https://local.theonion.com/mom-starting-to-fear-son-s-web-series-closest-thing-she-1819576697'

In [9]:
articles = []
for i in data:
    article = {}
#     article['link'] = i['article_link']
    article['headline'] = i['headline']
    article['is_sarcastic'] = i['is_sarcastic']
    articles.append(article)

In [10]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import seaborn as sns
import pandas as pd
import seaborn as sns
import numpy as np

pd.options.display.max_colwidth = 200

%matplotlib inline

In [11]:
ds = pd.DataFrame(articles)

In [13]:
ds.head()

,headline,is_sarcastic
0,former versace store clerk sues over secret 'black code' for minority shoppers,0
1,"the 'roseanne' revival catches up to our thorny political mood, for better and worse",0
2,mom starting to fear son's web series closest thing she will have to grandchild,1
3,"boehner just wants wife to listen, not come up with alternative debt-reduction ideas",1
4,j.k. rowling wishes snape happy birthday in the most magical way,0


### now we want to build up pipeline for NLP data.
1. vectorize headline
2. remove stop words
3. how do we handle new words?
4. can we use vader as corpus?
5. run classification models simple logistic L1/L2, Tree method, NN

In [14]:
corpus = list(ds['headline'])
sia = SentimentIntensityAnalyzer()

In [15]:
polarity_scores = [sia.polarity_scores(i) for i in corpus]


In [16]:
parsed_scores = []
for i in polarity_scores:
    parse_score = {}
    parse_score['compound'] = i['compound']
    parse_score['neg'] = i['neg']
    parse_score['neu'] = i['neu']
    parse_score['pos'] = i['pos']
    parsed_scores.append(parse_score)

In [17]:
ds['compound'] = [i['compound'] for i in polarity_scores]
ds['neg'] = [i['neg'] for i in polarity_scores]
ds['neu'] = [i['neu'] for i in polarity_scores]
ds['pos'] = [i['pos'] for i in polarity_scores]

In [18]:
ds.head()

,headline,is_sarcastic,compound,neg,neu,pos
0,former versace store clerk sues over secret 'black code' for minority shoppers,0,0.0000,0.000,1.000,0.000
1,"the 'roseanne' revival catches up to our thorny political mood, for better and worse",0,-0.3182,0.272,0.576,0.152
2,mom starting to fear son's web series closest thing she will have to grandchild,1,-0.4939,0.198,0.802,0.000
3,"boehner just wants wife to listen, not come up with alternative debt-reduction ideas",1,0.0000,0.000,1.000,0.000
4,j.k. rowling wishes snape happy birthday in the most magical way,0,0.6486,0.000,0.629,0.371


In [23]:
X = ds['headline'] # for X make sure single brackets
y = ds['is_sarcastic']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42,stratify=y)

In [42]:
pipe = Pipeline([
    ('cv', CountVectorizer(min_df=40,ngram_range=(1,4))),
#     ('tfidf', TfidfVectorizer()),
    ('lr',LogisticRegressionCV())
])

In [43]:
pipe.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('cv', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=40,
        ngram_range=(1, 4), preprocessor=None, stop_words=None,
        strip_...2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0))])

In [44]:
pipe.score(X_train,y_train), pipe.score(X_test,y_test)

(0.8164345264839499, 0.793800539083558)

### now we have working model can we:
1. add bigrams
2. run a random forest
3. try a NN